In [ ]:
# Default parameters
CATEGORIES = ["Apparel", "Baby"]
FROM_DATE = "2015-08-28"
TO_DATE = "2015-08-30"
OUTPUT_LOCATION = "s3://vasveena-test-demo/mwaa/query_output/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1645574661218_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for CATEGORY in CATEGORIES:
    sql = (
        f"CREATE EXTERNAL TABLE IF NOT EXISTS {CATEGORY}"
        "(review_id STRING,product_id STRING,product_title STRING,star_rating INT,verified_purchase STRING,review_date DATE)"
        f"STORED AS PARQUET LOCATION 's3://amazon-reviews-pds/parquet/product_category={CATEGORY}/'"
    )

    spark.sql(sql)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]
DataFrame[]

In [ ]:
spark.sql("show tables").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  apparel|      false|
| default|     baby|      false|
+--------+---------+-----------+

In [ ]:
for CATEGORY in CATEGORIES:
    sql = (
        f"SELECT product_title, AVG(star_rating), count(review_id) AS review_count FROM {CATEGORY} "
        f"WHERE review_date >= '{FROM_DATE}' AND review_date <= '{TO_DATE}' AND verified_purchase='Y' "
        "GROUP BY product_title "
        "ORDER BY SUM(star_rating) desc "
        "limit 20"
    )

    df = spark.sql(sql)
    output_folder = f"{OUTPUT_LOCATION}/{CATEGORY}_{FROM_DATE}_{TO_DATE}"
    df.write.csv(output_folder, mode="overwrite")
    print(f"Saved query result to {output_folder}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saved query result to s3://vasveena-test-demo/mwaa/query_output//Apparel_2015-08-28_2015-08-30
Saved query result to s3://vasveena-test-demo/mwaa/query_output//Baby_2015-08-28_2015-08-30